<a href="https://colab.research.google.com/github/chimaOkwuokei/ISEND-PAU/blob/main/FCN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
from torchvision import models
from torch.hub import load_state_dict_from_url

from PIL import Image
import cv2
import numpy as np
from matplotlib import pyplot as plt

In [3]:
if __name__ == "__main__":

    # Read ImageNet class id to name mapping
    with open('imagenet_classes.txt') as f:
        labels = [line.strip() for line in f.readlines()]

In [7]:
from torchvision import transforms

# Read image
original_image = cv2.imread('camel.jpg')

# Convert original image to RGB format
image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)

# Transform input image
# 1. Convert to Tensor
# 2. Subtract mean
# 3. Divide by standard deviation

transform = transforms.Compose([
             transforms.ToTensor(), #Convert image to tensor.
             transforms.Normalize(
             mean=[0.485, 0.456, 0.406],   # Subtract mean
             std=[0.229, 0.224, 0.225]     # Divide by standard deviation
             )])

image = transform(image)
image = image.unsqueeze(0)
print(transform)
image

Compose(
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
)


tensor([[[[ 0.1426,  0.1597,  0.1597,  ...,  0.5536,  0.5364,  0.5364],
          [ 0.1597,  0.1768,  0.1768,  ...,  0.5707,  0.5707,  0.5707],
          [ 0.1597,  0.1768,  0.1939,  ...,  0.5878,  0.5878,  0.5878],
          ...,
          [-1.9467,  0.6049,  2.0434,  ...,  0.4166,  1.1358,  0.8618],
          [-1.9467,  0.5878,  2.0434,  ...,  0.6906, -0.0458,  0.1083],
          [-1.9980,  0.4679,  1.9407,  ..., -0.3369,  0.5536, -0.1314]],

         [[ 1.0455,  1.0630,  1.0630,  ...,  1.2381,  1.2206,  1.2206],
          [ 1.0630,  1.0805,  1.0805,  ...,  1.2556,  1.2556,  1.2556],
          [ 1.0630,  1.0805,  1.0980,  ...,  1.2731,  1.2731,  1.2731],
          ...,
          [-1.9832,  0.6078,  2.0084,  ..., -0.0399,  0.6779,  0.3978],
          [-1.9832,  0.5903,  2.0084,  ...,  0.2402, -0.5301, -0.3725],
          [-2.0357,  0.4678,  1.9034,  ..., -0.8102,  0.0826, -0.6176]],

         [[ 1.8731,  1.8905,  1.8905,  ...,  1.7685,  1.7511,  1.7511],
          [ 1.8905,  1.9080,  

In [9]:
import torch
import torch.nn as nn
from torchvision import models
from torch.hub import load_state_dict_from_url

from PIL import Image
import cv2
import numpy as np
from matplotlib import pyplot as plt

from torchvision import transforms
from torchsummary import summary


# Define the architecture by modifying resnet.
# Original code is here
# https://github.com/pytorch/vision/blob/b2e95657cd5f389e3973212ba7ddbdcc751a7878/torchvision/models/resnet.py
class FullyConvolutionalResnet18(models.ResNet):
    def __init__(self, num_classes=1000, pretrained=False, **kwargs):

        # Start with standard resnet18 defined here
        # https://github.com/pytorch/vision/blob/b2e95657cd5f389e3973212ba7ddbdcc751a7878/torchvision/models/resnet.py
        super().__init__(block=models.resnet.BasicBlock, layers=[2, 2, 2, 2], num_classes=num_classes, **kwargs)
        if pretrained:
            state_dict = load_state_dict_from_url(models.resnet.model_urls["resnet18"], progress=True)
            self.load_state_dict(state_dict)

        # Replace AdaptiveAvgPool2d with standard AvgPool2d
        # https://github.com/pytorch/vision/blob/b2e95657cd5f389e3973212ba7ddbdcc751a7878/torchvision/models/resnet.py#L153-L154
        self.avgpool = nn.AvgPool2d((7, 7))

        # Add final Convolution Layer.
        self.last_conv = torch.nn.Conv2d(in_channels=self.fc.in_features, out_channels=num_classes, kernel_size=1)
        self.last_conv.weight.data.copy_(self.fc.weight.data.view(*self.fc.weight.data.shape, 1, 1))
        self.last_conv.bias.data.copy_(self.fc.bias.data)

    # Reimplementing forward pass.
    # Replacing the following code
    # https://github.com/pytorch/vision/blob/b2e95657cd5f389e3973212ba7ddbdcc751a7878/torchvision/models/resnet.py#L197-L213
    def _forward_impl(self, x):
        # Standard forward for resnet18
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)

        # Notice, there is no forward pass
        # through the original fully connected layer.
        # Instead, we forward pass through the last conv layer
        x = self.last_conv(x)
        return x


if __name__ == "__main__":

    # Read ImageNet class id to name mapping
    with open('imagenet_classes.txt') as f:
        labels = [line.strip() for line in f.readlines()]

    # Read image
    original_image = cv2.imread('camel.jpg')

    # Convert original image to RGB format
    image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)

    # Transform input image
    # 1. Convert to Tensor
    # 2. Subtract mean
    # 3. Divide by standard deviation

    transform = transforms.Compose([
                 transforms.ToTensor(), #Convert image to tensor.
                 transforms.Normalize(
                 mean=[0.485, 0.456, 0.406],   # Subtract mean
                 std=[0.229, 0.224, 0.225]     # Divide by standard deviation
                 )])

    image = transform(image)
    image = image.unsqueeze(0)

    # Load modified resnet18 model with pretrained ImageNet weights
    model = FullyConvolutionalResnet18(pretrained=True).eval()


    with torch.no_grad():
        # Perform inference.
        # Instead of a 1x1000 vector, we will get a
        # 1x1000xnxm output ( i.e. a probabibility map
        # of size n x m for each 1000 class,
        # where n and m depend on the size of the image.)
        preds = model(image)
        preds = torch.softmax(preds, dim=1)

        print('Response map shape : ', preds.shape)

        # Find the class with the maximum score in the n x m output map
        pred, class_idx = torch.max(preds, dim=1)
        print(class_idx)


        row_max, row_idx = torch.max(pred, dim=1)
        col_max, col_idx = torch.max(row_max, dim=1)
        predicted_class = class_idx[0, row_idx[0, col_idx], col_idx]

        # Print top predicted class
        print('Predicted Class : ', labels[predicted_class], predicted_class)

        # Find the n x m score map for the predicted class
        score_map = preds[0, predicted_class, :, :].cpu().numpy()
        score_map = score_map[0]

        # Resize score map to the original image size
        score_map = cv2.resize(score_map, (original_image.shape[1], original_image.shape[0]))

        # Binarize score map
        _, score_map_for_contours = cv2.threshold(score_map, 0.25, 1, type=cv2.THRESH_BINARY)
        score_map_for_contours = score_map_for_contours.astype(np.uint8).copy()

        # Find the countour of the binary blob
        contours, _ = cv2.findContours(score_map_for_contours, mode=cv2.RETR_EXTERNAL, method=cv2.CHAIN_APPROX_SIMPLE)

        # Find bounding box around the object.
        rect = cv2.boundingRect(contours[0])

        # Apply score map as a mask to original image
        score_map = score_map - np.min(score_map[:])
        score_map = score_map / np.max(score_map[:])

        score_map = cv2.cvtColor(score_map, cv2.COLOR_GRAY2BGR)
        masked_image = (original_image * score_map).astype(np.uint8)

        # Display bounding box
        cv2.rectangle(masked_image, rect[:2], (rect[0] + rect[2], rect[1] + rect[3]), (0, 0, 255), 2)

        # Display images
        cv2.imshow("Original Image", original_image)
        cv2.imshow("scaled_score_map", score_map)
        cv2.imshow("activations_and_bbox", masked_image)
        cv2.waitKey(0)



AttributeError: module 'torchvision.models.resnet' has no attribute 'model_urls'

In [8]:
# Load modified resnet18 model with pretrained ImageNet weights
# model = FullyConvolutionalResnet18(pretrained=True).eval()

NameError: name 'FullyConvolutionalResnet18' is not defined

In [18]:
import torch
import torch.nn as nn
from torchvision import models
from PIL import Image
import cv2
import numpy as np
from matplotlib import pyplot as plt
from torchvision import transforms

# Function to load the pre-trained FCN ResNet50 model
def load_fcn_resnet50(pretrained=True, num_classes=1000):
    model = models.segmentation.fcn_resnet50(pretrained=pretrained)
    # If you want to adjust for a different number of classes, modify the final classifier
    if num_classes != 1000:
        model.classifier[4] = nn.Conv2d(512, num_classes, kernel_size=1)
    return model


if __name__ == "__main__":

    # Read ImageNet class id to name mapping
    with open('imagenet_classes.txt') as f:
        labels = [line.strip() for line in f.readlines()]

    # Read image
    original_image = cv2.imread('camel.jpg')

    # Convert original image to RGB format
    image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)

    # Transform input image
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

    image = transform(image)
    image = image.unsqueeze(0)

    # Load FCN-ResNet50 model with pretrained weights
    model = load_fcn_resnet50(pretrained=True).eval()




In [19]:
with torch.no_grad():
    # Perform inference
    output = model(image)

    # Extract the prediction map from the output (it is stored in the 'out' key)
    preds = output['out']

    # Apply softmax to the output tensor along the class dimension
    preds = torch.softmax(preds, dim=1)

    print('Response map shape : ', preds.shape)

    # Get the predicted class per pixel
    pred = torch.argmax(preds, dim=1).squeeze(0).cpu().numpy()

    # Use the predicted class to create a mask
    predicted_class = np.unique(pred)[0]  # Assume the dominant class in the image

    print(f'Predicted Class: {labels[predicted_class]}')

    # Find the score map for the predicted class
    score_map = preds[0, predicted_class, :, :].cpu().numpy()


Response map shape :  torch.Size([1, 21, 725, 1920])
Predicted Class: tench, Tinca tinca
